In [3]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from urllib.request import urlopen



def read_image(image: Image.Image):
    image = image.convert("RGB")
    img_numpy = np.array(image).astype(np.float32)
    img_numpy = img_numpy.transpose(2, 0, 1)
    img_numpy = np.expand_dims(img_numpy, axis=0)
    return img_numpy


providers = [
    (
        "TensorrtExecutionProvider",
        {
            "device_id": 0,
            "trt_max_workspace_size": 8589934592,
            "trt_fp16_enable": True,
            "trt_engine_cache_enable": True,
            "trt_engine_cache_path": "./trt_cache",
            "trt_force_sequential_engine_build": False,
            "trt_max_partition_iterations": 10000,
            "trt_min_subgraph_size": 1,
            "trt_builder_optimization_level": 5,
            "trt_timing_cache_enable": True,
        },
    ),
]

session = ort.InferenceSession("merged.onnx", providers=providers)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


img = Image.open(
    urlopen(
        "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png"
    )
)


output = session.run([output_name], {input_name: read_image(img)})

env: LD_LIBRARY_PATH=/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib/python3.11/site-packages/tensorrt_libs:/home/dnth/mambaforge-pypy3/envs/ram-onnx-inference/lib:$LD_LIBRARY_PATH
*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/python/onnxruntime_pybind_state.cc:490 void onnxruntime::python::RegisterTensorRTPluginsAsCustomOps(PySessionOptions&, const onnxruntime::ProviderOptions&) Please install TensorRT libraries as mentioned in the GPU requirements page, make sure they're in the PATH or LD_LIBRARY_PATH, and that your GPU is supported.
 when using [('TensorrtExecutionProvider', {'device_id': 0, 'trt_max_workspace_size': 8589934592, 'trt_fp16_enable': True, 'trt_engine_cache_enable': True, 'trt_engine_cache_path': './trt_cache', 'trt_force_sequential_engine_build': False, 'trt_max_partition_iterations': 10000, 'trt_min_subgraph_size': 1, 'trt_builder_optimization_level': 5, 'trt_timing_cache_enable': True})]
Falling back to ['CPUExecutionProvider

2024-09-11 16:22:28.688929463 [E:onnxruntime:Default, provider_bridge_ort.cc:1978 TryGetProviderInfo_TensorRT] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libcudnn.so.9: cannot open shared object file: No such file or directory



In [4]:
output


[array([[-9.80133116e-02, -7.09748715e-02,  4.18799594e-02,
          3.05471003e-01,  8.56352389e-01, -6.08867884e-01,
         -1.14489764e-01, -5.66096365e-01, -3.39009434e-01,
         -4.52989995e-01, -4.44207609e-01, -1.95399925e-01,
         -2.15590537e-01, -5.99971041e-03,  8.06181803e-02,
         -5.37951738e-02, -1.50203824e-01,  1.59872174e-02,
         -1.08945347e-01, -5.41491270e-01, -3.88307571e-01,
         -1.11853182e-01, -1.48164034e-01,  2.78708965e-01,
         -4.75473583e-01,  1.37983754e-01, -1.65390164e-01,
         -5.86463869e-01,  1.19677067e-01, -3.23068619e-01,
         -2.87177920e-01, -3.55236679e-01, -1.68291748e-01,
          3.55988741e-03,  1.81204945e-01, -9.62484926e-02,
         -3.28296900e-01,  2.45878473e-03,  5.91688305e-02,
          5.88691309e-02,  3.47869545e-01, -1.48637593e-02,
          4.14957553e-02, -2.67596275e-01,  1.97183609e-01,
         -2.29016870e-01,  5.51088750e-01,  1.20698214e-02,
         -8.33828747e-02, -2.96069771e-0

In [5]:
output[0].shape

(1, 1000)

In [6]:
import torch
output = torch.from_numpy(output[0])
print(output.shape)

torch.Size([1, 1000])


In [9]:
from imagenet_classes import IMAGENET2012_CLASSES

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=10)

im_classes = list(IMAGENET2012_CLASSES.values())
class_names = [im_classes[i] for i in top5_class_indices[0]]

# Print class names and probabilities
for name, prob in zip(class_names, top5_probabilities[0]):
    print(f"{name}: {prob:.2f}%")

espresso: 33.73%
cup: 2.07%
chocolate sauce, chocolate syrup: 1.30%
bakery, bakeshop, bakehouse: 0.98%
coffee mug: 0.86%
eggnog: 0.77%
tray: 0.67%
ice cream, icecream: 0.62%
restaurant, eating house, eating place, eatery: 0.53%
plate: 0.47%
